In [1]:
import numpy as np
import pandas as pd
from openfisca_survey_manager.survey_collections import SurveyCollection
from openfisca_survey_manager import default_config_files_directory as config_files_directory

In [ ]:
# Step_1_1_build_dataframes_menages

def load_data_menages_bdf_entd(year_data):
    # Load ENTD data :
    year_entd = 2019

    entd_survey_collection = SurveyCollection.load(
        collection = 'enquete_transports', config_files_directory = config_files_directory
        )
    survey_entd = entd_survey_collection.get_survey('enquete_transports_{}'.format(year_entd))

    input_entd_tcm_menage = survey_entd.get_values(table = 'tcm_men_public_V2')
    input_entd_menage = survey_entd.get_values(table = 'q_menage_public_V2')

    # Load BdF data :
    year_bdf = year_data
    openfisca_survey_collection = SurveyCollection.load(collection = 'openfisca_indirect_taxation')
    openfisca_survey = openfisca_survey_collection.get_survey('openfisca_indirect_taxation_data_{}'.format(year_bdf))
    input_bdf = openfisca_survey.get_values(table = 'input')
    input_bdf.reset_index(inplace = True)

    # Create variable for total spending
    liste_variables = input_bdf.columns.tolist()
    postes_agreges = ['poste_{}'.format(index) for index in
        ['0{}'.format(i) for i in range(1, 10)] + ['10', '11', '12']
        ]
    input_bdf['depenses_tot'] = 0
    for element in liste_variables:
        for poste in postes_agreges:
            if element[:8] == poste:
                input_bdf['depenses_tot'] += input_bdf[element]
                    
    # Set variables :
    if year_entd == 2008:
        variables_tcm_menage_entd = [
            'ident_men',
            'agepr',
            'cs42pr',
            'dip14pr',
            'nactifs',
            'nbuc',             # ocde10
            'nenfants',
            'nivie10',          # déciles de revenu par uc
            'npers',
            'numcom_au2010',    # cataeu
            'pondv1',           # poids ménage
            'revuc',            # revenus simulés par UC
            'rlog',             # allocations logement
            'situapr',
            'tu99',
            'tau99',
            # 'typlog',
            'typmen5',          # type ménage
            # To be completed
            ]

        variables_menage_entd = [
            'ident_men',
            'v1_logpiec',       # nombre de pièces dans logement
            'v1_logocc',        # statut d'occupation du logement
            'v1_logloymens',    # loyer mensuel calculé
            ]
    if year_entd == 2019:
        variables_tcm_menage_entd = [
            'ident_men',
            'agepr',
            'cs42pr',
            'dipdetpr',
            'nactifs',
            'coeffuc',             # ocde10
            'nenfants',
            'decile_rev_uc',        # déciles de revenu par uc
            'npers',
            #'numcom_au2010',       # cataeu
            'pond_menc',            # poids ménage
            #'rev_final_uc',        # revenus simulés par UC   (pas dispo dans la table public demander table chercheur)
            #'rlog',                # allocations logement     (pas dispo dans la table public demander table chercheur)
            'situapr',
            'tuu2017_res',       
            #'tau99',               (pas dispo dans la table public demander table chercheur)
            # 'typlog',
            'typmen5',              # type ménage
            # To be completed
            ]
        
        variables_menage_entd = [
            'ident_men']
        
    variables_menage_bdf = [
        'ident_men',
        'aidlog1',
        'aidlog2',
        'agepr',            # âge de la pr
        'cataeu',
        'cs42pr',
        'dip14pr',
        'mloy_d',           # loyer mensuel
        'nactifs',
        'nbphab',
        'nenfants',
        'npers',
        'ocde10',           # nb unités de conso
        'pondmen',
        'poste_07_2_2_1',   # modifié Hervé 25/04 (07_2_2_1_1 -> 07_2_2_1)
        'revtot',           # revenu total
        'situapr',
        'stalog',
        'tau',
        'tuu',
        # 'typlog',
        'typmen',
        # To be completed
        ]

    # Keep relevant variables :
    tcm_menage_entd_keep = input_entd_tcm_menage[variables_tcm_menage_entd]
    menage_entd_keep = input_entd_menage[variables_menage_entd]
    menage_bdf_keep = input_bdf[variables_menage_bdf]

    # Merge entd tables into one dataframe
    data_entd = tcm_menage_entd_keep.merge(menage_entd_keep, on = 'ident_men')

    del input_entd_tcm_menage, input_entd_menage, input_bdf

    return data_entd, menage_bdf_keep

In [30]:
# Step_1_2_build_dataframes_vehicles
def load_data_vehicules_bdf_entd(year_data):
    year_entd = 2019

    entd_survey_collection = SurveyCollection.load(
        collection = 'enquete_transports', config_files_directory = config_files_directory
        )
    survey_entd = entd_survey_collection.get_survey('enquete_transports_{}'.format(year_entd))
    input_entd_vehicule = survey_entd.get_values(table = 'q_voitvul_public_V2')
    input_entd_menage = survey_entd.get_values(table = 'q_menage_public_V2')

    # Load BdF data :

    year_bdf = year_data

    openfisca_survey_collection = SurveyCollection.load(collection = 'budget_des_familles')
    openfisca_survey = openfisca_survey_collection.get_survey('budget_des_familles_{}'.format(year_bdf))
    input_bdf = openfisca_survey.get_values(table = 'AUTOMOBILE')
    input_bdf.reset_index(inplace = True)

    variables_vehicules_entd = [
        'ident_men',        # identifiant du ménage
        'ident_numveh',     # identifiant du véhicule
        'num_veh',          # numéro du véhicule
        'energie_agrege',   # carburant principal
        'puis_fisc_fin',    # puissance fiscale 
        'age',              # age du véhicule (en mois)
        #'v1_rang_veh',     # rang du véhicule dans le ménage
        #'v1_sfvachat',     # état du véhicule à l'achat
        'kvcons',           # consommation
        #'v1_sfvdatcg',     # date carte grise
        'annee_1mec',       # Année mise en circulation
        'kvkm1anv'          # nb kilomètres 12 derniers mois, redressés
        ]

    variables_menage_entd = [
        'ident_men', 
        #'v1_logdist01',     # distance commerces et supermarchés                       (pas dans q_menage regarder ailleurs ?)
        #'v1_logdist15',     # distance arrêts transports en commun                     (pas dans q_menage regarder ailleurs ?)
        'jnbveh',            # nb de voitures particulières
        #'v1_jpasvoit_b',    # utilité du VP : se rendre au travail                     (pas dans q_menage regarder ailleurs ?)
        #'v1_jpasvoit_c',    # utilité du VP : déplacements dans le cadre du travail    (pas dans q_menage regarder ailleurs ?)
        ]

    if year_bdf == 2011:
        variables_vehicules_bdf = [
            'ident_men',
            'acqvoi',           # état du véhicule à l'achat
            'anvoi',            # année du véhicule
            'carbu',            # carburant utilisé
            'expvoi1',          # est utilisé pour les trajets domicile-travail
            'expvoi2',          # est utilisé pour les déplacements professionnels
            'km_auto',
            'nbvehic',          # nb de véhicules du ménage
            'numvehic',         # numéro du véhicule
            'privoi_d',         # prix à l'achat, redressé
            'recvoi',           # année d'achat
            ]
    if year_bdf == 2017:
        variables_vehicules_bdf = [
            'ident_men',
            'acqvoi',           # état du véhicule à l'achat
            'anvoi',            # année du véhicule
            'carbu1',           # le carburant utilisé est l'essence
            'carbu2',           # diesel
            'carbu3',           # GPL
            'carbu4',           # electrique
            'carbu5',           # autre type de carburant
            'expvoi1',          # est utilisé pour les trajets domicile-travail
            'expvoi2',          # est utilisé pour les déplacements professionnels
            'km_auto',
            'nbvehic',          # nb de véhicules du ménage
            'numvehic',         # numéro du véhicule
            'privoi_d',         # prix à l'achat, redressé
            'recvoi',           # année d'achat
        ]

    # Keep relevant variables :
    vehicule_menage_entd_keep = input_entd_vehicule[variables_vehicules_entd]
    menage_entd_keep = input_entd_menage[variables_menage_entd]
    vehicule_bdf_keep = input_bdf[variables_vehicules_bdf]
    
    return vehicule_menage_entd_keep, menage_entd_keep, vehicule_bdf_keep

In [31]:
# Step_1_2_build_dataframes_vehicles
def merge_vehicule_menage(year_data):
    data_entd, data_entd_menage, data_bdf = load_data_vehicules_bdf_entd(year_data)

    # Calcul de l'âge du véhicule
    data_entd['anvoi'] = pd.to_numeric(data_entd['annee_1mec'], errors = 'coerce')

    data_entd['age_vehicule'] = 0
    data_entd.loc[data_entd['anvoi'] != 0, 'age_vehicule'] = 2019 - data_entd['anvoi']

    data_bdf['anvoi' ] = data_bdf['anvoi'].fillna(0).astype(int)
    data_bdf['age_vehicule'] = 0
    data_bdf.loc[data_bdf['anvoi'] != 0, 'age_vehicule'] = year_data - data_bdf['anvoi']

    # Définition des véhicules par carburant
    data_entd['essence'] = 0
    data_entd.loc[data_entd['energie_agrege'] == 1, 'essence'] = 1
    data_entd['diesel'] = 0
    data_entd.loc[data_entd['energie_agrege'] == 2, 'diesel'] = 1
    data_entd['autre_carbu'] = 0
    data_entd.loc[data_entd['energie_agrege'] > 2, 'autre_carbu'] = 1

    if year_data == 2017:
        carbu_cols = ['carbu1', 'carbu2', 'carbu3', 'carbu4', 'carbu5']
        data_bdf['carbu'] = data_bdf[carbu_cols].idxmax(axis=1).str.extract('(\d)').astype(int)
        data_bdf.drop(carbu_cols, axis = 1, inplace = True)

    data_bdf['essence'] = 0
    data_bdf.loc[data_bdf['carbu'] == 1, 'essence'] = 1
    data_bdf['diesel'] = 0
    data_bdf.loc[data_bdf['carbu'] == 2, 'diesel'] = 1
    data_bdf['autre_carbu'] = 0
    data_bdf.loc[data_bdf['carbu'] > 2, 'autre_carbu'] = 1

    # déf des distances parcourues par carburant
    data_entd['distance_essence'] = 0
    data_entd.loc[data_entd['essence'] == 1, 'distance_essence'] = data_entd['kvkm1anv']
    data_entd['distance_diesel'] = 0
    data_entd.loc[data_entd['diesel'] == 1, 'distance_diesel'] = data_entd['kvkm1anv']
    data_entd['distance_autre_carbu'] = 0
    data_entd.loc[data_entd['autre_carbu'] == 1, 'distance_autre_carbu'] = data_entd['kvkm1anv']

    # Df avec le nombre de véhicule et les distances pour chaque type de carburant
    data_vehicule_entd = data_entd[
        ['essence',
        'diesel',
        'autre_carbu',
        'distance_essence',
        'distance_diesel',
        'distance_autre_carbu',
        'ident_men',
            ]
        ].groupby(by = 'ident_men').sum()
    data_vehicule_entd = data_vehicule_entd.reset_index()

    # Df avec les infos du véhicule principal (dans ENTD)
    data_entd = data_entd.sort_values(by = 'kvkm1anv', ascending= False)
    data_entd = data_entd.drop_duplicates(['ident_men'], keep='first')
    data_entd.rename(
        columns = {
            'puis_fisc_fin': 'puissance',
            'kvcons': 'consommation',
            },
        inplace = True,
        )
    data_entd = data_entd[
        ['ident_men', 'puissance', 'consommation', 'age_vehicule']
        ]

    # déf des distances parcourues par carburant
    data_bdf['km_essence'] = 0
    data_bdf.loc[data_bdf['essence'] == 1, 'km_essence'] = data_bdf['km_auto']
    data_bdf['km_diesel'] = 0
    data_bdf.loc[data_bdf['diesel'] == 1, 'km_diesel'] = data_bdf['km_auto']
    data_bdf['km_autre_carbu'] = 0
    data_bdf.loc[data_bdf['autre_carbu'] == 1, 'km_autre_carbu'] = data_bdf['km_auto']

    # Df avec le nombre de véhicule et les distances pour chaque type de carburant
    data_vehicule_bdf = data_bdf[
        ['essence',
        'diesel',
        'autre_carbu',
        'km_essence',
        'km_diesel',
        'km_autre_carbu',
        'ident_men',
        ]
    ].groupby(by = 'ident_men').sum()
    data_vehicule_bdf = data_vehicule_bdf.reset_index()

    # Df avec les infos du véhicule principal (dans BdF)
    data_bdf = data_bdf.sort_values(by = ['nbvehic'])
    data_bdf = data_bdf.drop_duplicates(['ident_men'], keep='first')
    data_bdf.rename(
        columns = {
            'acqvoi': 'etat_veh_achat',
            'expvoi1': 'vp_domicile_travail',
            'expvoi2': 'vp_deplacements_pro',
            'nbvehic': 'veh_tot',
            'privoi_d': 'prix_achat',
            },
        inplace = True,
        )
    data_bdf = data_bdf[
        ['ident_men', 'prix_achat', 'veh_tot', 'etat_veh_achat', 'age_vehicule', 'vp_domicile_travail', 'vp_deplacements_pro']
    ]

    # Df infos comportements ménages
    data_entd_menage.rename(
        columns = {
            #'v1_logdist01': 'distance_commerces',              (pas dans q_menage regarder ailleurs ?)
            #'v1_logdist15': 'distance_transports_communs',     (pas dans q_menage regarder ailleurs ?)    
            'jnbveh': 'veh_tot',
            #'v1_jpasvoit_b': 'vp_domicile_travail',            (pas dans q_menage regarder ailleurs ?)
            #'v1_jpasvoit_c': 'vp_deplacements_pro'             (pas dans q_menage regarder ailleurs ?)
            },
        inplace = True,
        )


    # Merge les différentes df
    data_entd_full = data_vehicule_entd.merge(data_entd, on = 'ident_men', how = 'left')
    data_entd_final = data_entd_menage.merge(data_entd_full, on = 'ident_men', how = 'left')

    data_bdf_full = data_vehicule_bdf.merge(data_bdf, on = 'ident_men', how = 'left')
    
    return data_entd_final, data_bdf_full


In [32]:
# Step_1_2_build_dataframes_vehicles
def build_df_menages_vehicles(year_data):
    data_menages_entd, data_menages_bdf = load_data_menages_bdf_entd(year_data)

    data_vehicules_entd, data_vehicules_bdf = merge_vehicule_menage(year_data)
    data_vehicules_bdf['ident_men'] = data_vehicules_bdf['ident_men'].astype(str)

    data_entd = data_menages_entd.merge(data_vehicules_entd, on = 'ident_men')
    
    data_bdf = data_menages_bdf.merge(data_vehicules_bdf, on = 'ident_men', how = 'left')

    return data_entd, data_bdf

In [33]:
year_data = 2017

In [38]:
data_entd, data_bdf = build_df_menages_vehicles(year_data)

In [39]:
data_entd.columns

Index(['ident_men', 'agepr', 'cs42pr', 'dipdetpr', 'nactifs', 'coeffuc',
       'nenfants', 'decile_rev_uc', 'npers', 'pond_menc', 'situapr',
       'tuu2017_res', 'typmen5', 'veh_tot', 'essence', 'diesel', 'autre_carbu',
       'distance_essence', 'distance_diesel', 'distance_autre_carbu',
       'puissance', 'consommation', 'age_vehicule'],
      dtype='object')

In [40]:
#Step_2_homogenize_variable
def homogenize_variables_definition_bdf_entd(year_data):
    data_entd, data_bdf = build_df_menages_vehicles(year_data)
    check = data_bdf.query('aidlog1 != 0')
    assert (check['aidlog2'] == 0).any()

    data_bdf['aba'] = data_bdf['aidlog1'] + data_bdf['aidlog2']
    del check, data_bdf['aidlog1'], data_bdf['aidlog2']

    # Distribution des ménages parisiens dans les catégories des grands pôles
    #data_entd.loc[data_entd.numcom_au2010 == 113, 'numcom_au2010'] = 111
    #data_entd.loc[data_entd.numcom_au2010 == 114, 'numcom_au2010'] = 112

    data_bdf.loc[data_bdf.stalog == 5, 'stalog'] = 4
    data_bdf.loc[data_bdf.stalog == 6, 'stalog'] = 5

    # Rename
    data_entd.rename(
        columns = {
                'nbuc': 'ocde10',
                #'numcom_au2010': 'cataeu',
                'pondv1': 'pondmen',
                'pond_menc' : 'pondmen',
                'rlog': 'aba',
                'tau99': 'tau',
                'tu99': 'tuu',
                'tuu2017_res' : 'tuu',
                'typmen5': 'typmen',
                'v1_logloymens': 'mloy_d',
                'v1_logocc': 'stalog',
                'v1_logpiec': 'nbphab',
                },
        inplace = True,
    )

    data_entd = data_entd.sort_index(axis = 1)
    data_bdf = data_bdf.sort_index(axis = 1)

    return data_entd, data_bdf

In [ ]:
# step_2_homogenize_variables
def create_new_variables(year_data):
    data_entd, data_bdf = homogenize_variables_definition_bdf_entd(year_data)
    
    def create_new_variables_(data, option = None):
        assert option in ['entd', 'bdf']

        # Création de dummy variables pour la commune de résidence
        data['rural'] = 0
        data['petite_ville'] = 0
        data['moyenne_ville'] = 0
        data['grande_ville'] = 0
        data['paris'] = 0

        data.loc[data.tuu == 0, 'rural'] = 1
        data.loc[data.tuu == 1, 'petite_ville'] = 1
        data.loc[data.tuu == 2, 'petite_ville'] = 1
        data.loc[data.tuu == 3, 'petite_ville'] = 1
        data.loc[data.tuu == 4, 'moyenne_ville'] = 1
        data.loc[data.tuu == 5, 'moyenne_ville'] = 1
        data.loc[data.tuu == 6, 'moyenne_ville'] = 1
        data.loc[data.tuu == 7, 'grande_ville'] = 1
        data.loc[data.tuu == 8, 'paris'] = 1

        
        #data['aides_logement'] = 0
        #data.loc[data['aba'] == 1, 'aides_logement'] = 1
        #del data['aba']

        data['veh_tot'] = data['veh_tot'].fillna(0)
        #data['mloy_d'] = data['mloy_d'].fillna(0)

        data['part_essence'] = (
            data.essence / (data.essence + data.diesel + data.autre_carbu)
            )
        data['part_diesel'] = (
            data.diesel / (data.essence + data.diesel + data.autre_carbu)
            )
        data['nb_essence'] = data.veh_tot * data.part_essence
        data['nb_diesel'] = data.veh_tot * data.part_diesel
        data[['nb_essence', 'nb_diesel']] = data[['nb_essence', 'nb_diesel']].fillna(0)

        data.drop(
            ['part_essence', 'part_diesel', 'essence', 'diesel', 'autre_carbu'],
            axis = 1,
            inplace = True
            )

        if option == 'entd':
            data['distance'] = data.distance_diesel + data.distance_essence + data.distance_autre_carbu

        return data

    return create_new_variables_(data_entd, option = 'entd'), create_new_variables_(data_bdf, option = 'bdf')

In [53]:
data_entd, data_bdf = create_new_variables(year_data)

In [54]:
data_entd.columns

Index(['age_vehicule', 'agepr', 'coeffuc', 'consommation', 'cs42pr',
       'decile_rev_uc', 'dipdetpr', 'distance_autre_carbu', 'distance_diesel',
       'distance_essence', 'ident_men', 'nactifs', 'nenfants', 'npers',
       'pondmen', 'puissance', 'situapr', 'tuu', 'typmen', 'veh_tot', 'rural',
       'petite_ville', 'moyenne_ville', 'grande_ville', 'paris', 'nb_essence',
       'nb_diesel', 'distance'],
      dtype='object')

In [51]:
def create_niveau_vie_quantiles(year_data):
    data_entd, data_bdf = create_new_variables(year_data)

    def create_niveau_vie_quantiles_(data, option = None):
        assert option in ['entd', 'bdf']
        if option == 'entd':
            data['niveau_vie'] = data['revuc'].copy() * 12
            del data['revuc']
        else:
            data['niveau_vie'] = data.revtot / data.ocde10

        data = data.sort_values(by = ['niveau_vie'])
        data['sum_pondmen'] = data['pondmen'].cumsum()

        population_totale = data['sum_pondmen'].max()
        data['niveau_vie_decile'] = 0
        print(option)
        for j in range(1, 11):
            data.loc[data.sum_pondmen > population_totale * (float(j) / 10 - 0.1), 'niveau_vie_decile'] = j

        data['niveau_vie_quintile'] = 0
        for j in range(1, 6):
            data.loc[data.sum_pondmen > population_totale * (float(j) / 5 - 0.2), 'niveau_vie_quintile'] = j

        data = data.sort_index()
        del data['sum_pondmen']

        return data.copy()

    return create_niveau_vie_quantiles_(data_entd, option = 'entd'), create_niveau_vie_quantiles_(data_bdf, option = 'bdf')

In [52]:
create_niveau_vie_quantiles(year_data)

KeyError: 'revuc'